In [1]:
import os
import cv2
import numpy as np
import ipywidgets as ipyw
import lz4.frame
import pandas as pd

import matplotlib

from matplotlib import pyplot as plt
%matplotlib inline

from werdich_cfr.utils.visuals import ImageSliceViewer3D

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [193]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
meta_date = '200208'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+meta_date)
cfr_meta_file = 'tfr_files_dset_BWH_'+meta_date+'.parquet'
meta_df = pd.read_parquet(os.path.join(meta_dir, cfr_meta_file))

# Filter some stuff
meta_a4c = meta_df[meta_df.max_view == 'a4c']

In [194]:
print('Total number of files {}'.format(len(meta_a4c.filename.unique())))
print('Files with deltaX<0   {}'.format(len(meta_a4c[meta_a4c.deltaX<0].filename.unique())))
print('Files with deltaX=1.0 {}'.format(len(meta_a4c[meta_a4c.deltaX==1].filename.unique())))
print('Files with deltaY<0   {}'.format(len(meta_a4c[meta_a4c.deltaY<0].filename.unique())))
print('Files with deltaY=1.0 {}'.format(len(meta_a4c[meta_a4c.deltaY==1].filename.unique())))

Total number of files 7339
Files with deltaX<0   53
Files with deltaX=1.0 49
Files with deltaY<0   0
Files with deltaY=1.0 49


In [195]:
# Filter some mote values
meta_a4c = meta_df[(meta_df.max_view == 'a4c') & 
                   (0<meta_df.deltaX) & (meta_df.deltaX<1) &
                   (0<meta_df.deltaY) & (meta_df.deltaY<1)]

print('Total number of files {}'.format(len(meta_a4c.filename.unique())))
print('Files with deltaX<0   {}'.format(len(meta_a4c[meta_a4c.deltaX<0].filename.unique())))
print('Files with deltaX=1.0 {}'.format(len(meta_a4c[meta_a4c.deltaX==1].filename.unique())))
print('Files with deltaY<0   {}'.format(len(meta_a4c[meta_a4c.deltaY<0].filename.unique())))
print('Files with deltaY=1.0 {}'.format(len(meta_a4c[meta_a4c.deltaY==1].filename.unique())))

Total number of files 7220
Files with deltaX<0   0
Files with deltaX=1.0 0
Files with deltaY<0   0
Files with deltaY=1.0 0


In [196]:
# Pick a few files with small scaling factors
xs = meta_a4c[(0<meta_a4c.deltaX) & (meta_a4c.deltaX<0.022)]
xs_files = [os.path.join(xs[xs.filename==file].dir.values[0], file) for file in xs.filename.unique()]
print(xs.shape)
# And some with a larger scaling factor
xl = meta_a4c[(0.085<meta_a4c.deltaX) & (meta_a4c.deltaX<1.0)]
print(xl.shape)
xl_files = [os.path.join(xl[xl.filename==file].dir.values[0], file) for file in xl.filename.unique()]

(6, 52)
(10, 52)


In [197]:
# Load videos
def load_data(file):
    try:
        with lz4.frame.open(file, 'rb') as fp:
            data = np.load(fp)
    except IOError as err:
        print('Could not open this file: {}\n {}'.format(file, err))

    return data

In [198]:
def im_scale(im):
    """ convert single images to uint8 """
    # We can do other things here: e.g. background subtraction or contrast enhancement
    im_scaled = np.uint8((im - np.amin(im))/(np.amax(im) - np.amin(im))*256)
    #im_scaled_eq = cv2.equalizeHist(im_scaled)
    return im_scaled

def data2imarray(im_data):
    """
    apply imscale function to np.array
    arg: im_array (frame, height, width)
    returns: im_array (height, width, frame)
    """
    im_data = np.squeeze(im_data)
    im_list = [im_scale(im_data[im]) for im in range(im_data.shape[0])]
    im_array = np.array(im_list, dtype=np.uint16)
    im_array = np.moveaxis(im_array, 0, -1)
    return im_array

In [208]:
# SCALE THE IMAGES BY FACTORS
def im_scale_factor(im,dx,dy):
    """ convert single images to uint8 """
    # We can do other things here: e.g. background subtraction or contrast enhancement
    im_scaled = np.uint8((im - np.amin(im))/(np.amax(im) - np.amin(im))*256)
    #im_scaled_eq = cv2.equalizeHist(im_scaled)
    if (dx is not None) & (dy is not None):
        width = int(np.round(im_scaled.shape[1]*10*dx))
        height = int(np.round(im_scaled.shape[0]*10*dy))
        im_resized = cv2.resize(im_scaled, (width, height), interpolation=cv2.INTER_LINEAR)
    else:
        im_resized = im_scaled
    return im_resized

def data2imarray_factor(im_data,dx,dy):
    """
    apply imscale function to np.array
    arg: im_array (frame, height, width)
    returns: im_array (height, width, frame)
    """
    im_data = np.squeeze(im_data)
    im_list = [im_scale_factor(im_data[im],dx,dy) for im in range(im_data.shape[0])]
    im_array = np.array(im_list, dtype=np.uint16)
    im_array = np.moveaxis(im_array, 0, -1)
    return im_array

In [222]:
#for file in xs_files:
file = xs_files[3]
filename = os.path.basename(file)
data = load_data(file)
deltaXs = meta_df[meta_df.filename == filename].deltaX.values[0]
deltaYs = meta_df[meta_df.filename == filename].deltaY.values[0]
im_array_xs = data2imarray(data)
im_array_xs_factor = data2imarray_factor(data,deltaXs, deltaYs)
#im_array_xs_factor = data2imarray_factor(data, None, None)
#print(im_array_xs.shape)
print(im_array_xs_factor.shape)
print('original: deltaXs {}, deltaYs {}'.format(deltaXs, deltaYs))

(115, 153, 24)
original: deltaXs 0.014972539618611336, deltaYs 0.014972543343901634


In [220]:
#for file in xl_files:
file=xl_files[2]
filename = os.path.basename(file)
data = load_data(file)
deltaXl = meta_df[meta_df.filename == filename].deltaX.values[0]
deltaYl = meta_df[meta_df.filename == filename].deltaY.values[0]
im_array_xl = data2imarray(data)
im_array_xl_factor = data2imarray_factor(data, deltaXl, deltaYl)
#im_array_xl_factor = data2imarray_factor(data, None, None)
print(im_array_xl_factor.shape)
print('deltaX {}, deltaY {}'.format(deltaXl, deltaYl))

(296, 413, 66)
deltaX 0.08595988538681948, deltaY 0.06876790830945559


In [223]:
# Show original images
ImageSliceViewer3D(im_array_xs_factor)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [219]:
ImageSliceViewer3D(im_array_xl_factor)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…